<a href="https://colab.research.google.com/github/lucas-morais/sistema-de-recomendacao-imoveis/blob/semana1/Analise_Exploratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de Preços de Imóveis**

Análise exploaratóras de dados de Imóveis na cidade do Rio de Janeiro do banco de dados da ***InsightPlace***

## **Dicionário de Dados**


[Base de dados - InsightPlaces](https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip)

### Dicionário de Dados - Anuncio

| Colunas         |Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

## **Preparando o ambiente**

###**Instalando Dependências**

In [1]:
!pip install pyspark -q

     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 46.0 MB/s 


###**Criando Sessão do Spark**

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession\
    .builder\
    .master('local[*]')\
    .appName('analise-exploratoria')\
    .getOrCreate()

spark

##**Preparando Dados**

###**Carregando Dados**

In [75]:
# Buscando dados
!wget -q "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip"

!unzip -q semana-1.zip 

replace dataset_bruto.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [138]:
# Lendo dados em dataframe spark

df = spark.read.json("dataset_bruto.json")

df.show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
|{0, [20], [15], [...|[{2de09d46-dc0d-4...|{3c7057f5-0923-42...|
|{3, [43], [43], [...|[{147a80d9-cd40-4...|{5a9736b5-aaa0-4a...|
|{2, [42], [42], [...|[{35740004-063d-4...|{ec48d96a-137c-49...|
|{0, [], [12], [0]...|[{6d3d2aec-c96f-4...|{dad7db63-e19c-44...|
|{1, [41], [41], [...|[{3d404069-418e-4...|{a845f35f-3ab3-46...|
|{3, [43], [43], [...|[{5b8a8de7-2088-4...|{8c8de313-45d5-4c...|
|{0, [10], [4], [0...|[{5385da42-8276-4...|{0b1d4ee0-2bc3-4c...|
|{0, [12], [12], [...|[{7

In [139]:
df.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

## **Análise Exploratória**

### **Analisando Estrutura de Dados**

####**Separando Colnua de  Anúncios**

- Apenas a coluna de anúncios fará parte da análise inicial

In [140]:
df_anuncio = df.select("anuncio.*")
df_anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [141]:
print(f'Qauntidade de linhas: {df_anuncio.count()}')
print(f'Quantidade de colunas: {len(df_anuncio.columns)}')

Qauntidade de linhas: 89083
Quantidade de colunas: 14


####**Analisando Variável 'tipo_uso'**

In [142]:
df_anuncio.groupBy('tipo_uso').count().withColumnRenamed('count', 'Total').show()

+-----------+-----+
|   tipo_uso|Total|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [143]:
from pyspark.sql import functions as f

df_anuncio\
    .groupBy('tipo_uso')\
    .count()\
    .withColumn('Freq', f.col('count')/df_anuncio.count())\
    .show()

+-----------+-----+--------------------+
|   tipo_uso|count|                Freq|
+-----------+-----+--------------------+
|  Comercial| 4542|0.050986158975337606|
|Residencial|84541|  0.9490138410246624|
+-----------+-----+--------------------+



O tipo de uso **Residencial** representa aproximadamente 95% dos dados.



####**Analisando Variável 'tipo_unidade'**



In [144]:
df_anuncio.groupBy('tipo_unidade').count().withColumnRenamed('count', 'Total').show()

+------------+-----+
|tipo_unidade|Total|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [145]:
df_anuncio\
    .groupBy('tipo_unidade')\
    .count()\
    .withColumn('Freq', f.col('count')/df_anuncio.count())\
    .show()

+------------+-----+------------------+
|tipo_unidade|count|              Freq|
+------------+-----+------------------+
|      Outros|11963|0.1342904931356151|
| Apartamento|66801|0.7498737132786278|
|        Casa|10319|0.1158357935857571|
+------------+-----+------------------+



O tipo de unidade **Apartamento** representa aproximadamente 75% dos dados.


####**Analisando Variável 'tipo_anuncio'**


In [146]:
df_anuncio.groupBy('tipo_anuncio').count().withColumnRenamed('count', 'Total').show()

+------------+-----+
|tipo_anuncio|Total|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [147]:
df_anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .withColumn('Freq', f.col('count')/df_anuncio.count())\
    .show()

+------------+-----+--------------------+
|tipo_anuncio|count|                Freq|
+------------+-----+--------------------+
|       Usado|88827|  0.9971262754958858|
|  Lançamento|  256|0.002873724504114141|
+------------+-----+--------------------+



O tipo anúncio **Usado** representa aproximadamente 99.7% dos dados

####**Transformando colunas do tipo array para inteiros**

In [148]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [149]:
from pyspark.sql.types import IntegerType

In [150]:
# Cria função para converter arrays em inteiros
def cast_array_column(array):
    if len(array) and (array[0] != None):
        return int(array[0])
    return None

In [151]:
# Cria udf
cast_array_udf = f.udf(lambda x: cast_array_column(x), IntegerType())

In [152]:
# Separando colunas
colunas_array_long = [
    'banheiros',
    'suites',
    'quartos',
    'vaga'
]

colunas_array_string = [
    'area_total',
    'area_util'
]

In [153]:
for c in colunas_array_long + colunas_array_string:
    df_anuncio = df_anuncio.withColumn(c, cast_array_udf(c))

In [154]:
df_anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [155]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

### **Simplificando Coluna Endereço**

- Persistir apenas as informações **zona** e **bairro**

In [156]:
df_anuncio.select('endereco').printSchema()

root
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)



In [157]:
df_anuncio = df_anuncio\
    .withColumn('zona', df_anuncio.endereco.zona)\
    .withColumn('bairro', df_anuncio.endereco.bairro)\
    .drop('endereco')

In [158]:
df_anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+--------------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        zona|        bairro|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+--------------+
|    0|      null|       16|        0|                  []|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|Zona Central|        Centro|
|    0|      null|       14|        0|                  []|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|Zona Central|        Centro|
|    0|      1026|     1026|        0|                  []|1

In [159]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- bairro: string (nullable = true)



In [160]:
df_anuncio.groupBy('zona').count().show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|15277|
|  Zona Oeste|49047|
|Zona Central| 2122|
|    Zona Sul|22621|
|            |   16|
+------------+-----+



In [161]:
df_anuncio.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   26|
|       Tomás Coelho|   45|
|              Gávea|  729|
|            Ipanema| 2304|
|           Realengo|  142|
|      Gardênia Azul|   16|
|      Bento Ribeiro|   94|
|              Rocha|   51|
|Vicente de Carvalho|  108|
|              Dumas|    1|
|         Manguinhos|    2|
|       Campo Grande| 2013|
|           Botafogo| 3849|
|        Jacarepaguá| 5102|
|               Leme|  247|
|       Padre Miguel|   39|
|  Engenho da Rainha|   33|
|       Santo Cristo|  403|
|           Flamengo| 1456|
| Pedra de Guaratiba|  106|
+-------------------+-----+
only showing top 20 rows



#### **Modificando informações da coluna valores**

In [162]:
df_anuncio = df_anuncio\
    .withColumn('condominio', df_anuncio.valores.condominio)\
    .withColumn('iptu', df_anuncio.valores.iptu)\
    .withColumn('tipo', df_anuncio.valores.tipo)\
    .withColumn('valor', df_anuncio.valores.valor)\
    .drop('valores')

In [163]:
df_anuncio.select('condominio', 'iptu', 'valor', 'tipo').show()

+----------+------+-------+-------+
|condominio|  iptu|  valor|   tipo|
+----------+------+-------+-------+
|     [260]| [107]|[10000]|[Venda]|
|     [260]| [107]|[10000]|[Venda]|
|    [null]|[1613]|[10000]|[Venda]|
|      [80]|[null]|[10000]|[Venda]|
|       [0]|   [0]| [5000]|[Venda]|
|     [223]|  [87]|[18500]|[Venda]|
|     [245]|[null]|[15000]|[Venda]|
|       [0]|   [0]|[15000]|[Venda]|
|     [250]|[null]|[20000]|[Venda]|
|       [0]|   [0]|[20000]|[Venda]|
|     [285]|[null]|[20000]|[Venda]|
|     [200]|  [86]|[11500]|[Venda]|
|     [470]|[null]|[12000]|[Venda]|
|     [294]|  [98]|[18000]|[Venda]|
|     [684]|[null]|[15000]|[Venda]|
|       [0]| [120]|[20000]|[Venda]|
|     [245]|[null]|[15000]|[Venda]|
|     [285]|[null]|[20000]|[Venda]|
|     [250]|[null]|[15000]|[Venda]|
|     [245]|[null]|[15000]|[Venda]|
+----------+------+-------+-------+
only showing top 20 rows



In [164]:
valores = ['condominio', 'iptu', 'valor']

for c in valores:
    df_anuncio = df_anuncio.withColumn(c, cast_array_udf(c))

In [171]:
df_anuncio = df_anuncio\
    .withColumn('tipo_venda', f.col('tipo').getItem(0))\
    .drop('tipo')

In [172]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- zona: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- valor: integer (nullable = true)
 |-- tipo_venda: string (nullable = true)



#### **Removendo registros de imóveis para alugar**

In [175]:
df_anuncio.groupBy('tipo_venda').count().show()

+----------+-----+
|tipo_venda|count|
+----------+-----+
|   Aluguel|  811|
|     Venda|88272|
+----------+-----+



In [180]:
df_anuncio = df_anuncio.select("*").where("tipo_venda == 'Venda'")

In [181]:
df_anuncio.groupBy('tipo_venda').count().show()

+----------+-----+
|tipo_venda|count|
+----------+-----+
|     Venda|88272|
+----------+-----+



In [182]:
df_anuncio = df_anuncio.drop('tipo_venda')

In [183]:
df_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- zona: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- valor: integer (nullable = true)



## **Salvando Dados**

### Salvando dados no formato parquet

In [184]:
df_anuncio.write.parquet('anuncios.parquet')

#### Salvando dados no formato csv

In [192]:
df_anuncio.select('caracteristicas')\
    .withColumn('caracteristicas', f.concat_ws(', ',f.col('caracteristicas')))\
    .show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|caracteristicas                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                             |
|                                                                                                                             |
|                                                                                                                             |
|Portão eletrônico, Condomínio fechado                                                                                        |
|                                                                                                       

In [193]:
df_anuncio_csv = df_anuncio.select('*')\
    .withColumn('caracteristicas', f.concat_ws(', ',f.col('caracteristicas')))

###

In [196]:
%%time
1 + 5

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs


6

### **Comparando Leitura: Parquet x CSV**

In [202]:
import time

start = time.time()
df_csv = spark.read.csv('anuncios.csv')
tempo_csv = time.time() - start

In [205]:
start = time.time()
df_parquet = spark.read.parquet('anuncios.parquet')
tempo_parquet = time.time() - start

In [206]:
print(f'Tempo de leitura csv: {tempo_csv}')
print(f'Tempo de leitura parquet: {tempo_parquet}')

Tempo de leitura csv: 0.4309980869293213
Tempo de leitura parquet: 0.16490793228149414


In [208]:
df_parquet.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- zona: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- valor: integer (nullable = true)



In [209]:
df_parquet.show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+------------+--------------+----------+----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|        zona|        bairro|condominio|iptu|valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+------------+--------------+----------+----+-----+
|    0|      null|       16|        0|                  []|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|Zona Central|        Centro|       260| 107|10000|
|    0|      null|       14|        0|                  []|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|Zona Central|        Centro|       260| 107|10000|
|    0|      1026|     1026|        0|                 